我们将使用 LlamaIndex 实现一个结合关键字和向量搜索检索器的自定义检索器，并且使用 Gemini大模型来进行多个文档聊天。

# 安装所需的包

在我们的例子中，使用 LlamaIndex 来构建自定义检索器，使用 Gemini 来构建嵌入模型和LLM推理，并使用 PyPDF 来构建数据连接器，因此，需要安装所需的库。

In [ ]:
!pip install llama-index
!pip install llama-index-multi-modal-llms-gemini
!pip install llama-index-embeddings-gemini

  Using cached llama_index_embeddings_gemini-0.1.8-py3-none-any.whl (2.9 kB)


# 设置Google API密钥

利用 Google Gemini 作为大型语言模型来生成响应，并作为嵌入模型，使用 LlamaIndex 将数据转换和存储在vector数据库或内存中。

如果没有Google API Key，可以在这里（http://ai.google.dev/）申请。

In [ ]:
from getpass import getpass

GOOGLE_API_KEY = getpass("Enter your Google API:")

Enter your Google API:··········


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
path = "/content/drive/My Drive/Colab/RAG/LIamaIndex_RAG"
os.chdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 加载数据并创建文档节点

在 LlamaIndex 中，数据加载是使用 SimpleDirectoryLoader 完成的。首先，需要创建一个文件夹并将任何格式的数据上传到此数据文件夹中。在我们的示例中，我们将把一个 PDF 文件上传到 data 文件夹中。加载文档后，将文档拆分为更小的段，将其解析为节点。节点是在 LlamaIndex 框架中定义的数据架构。

最新版本的 LlamaIndex 更新了其代码结构，现在包括节点解析器、嵌入模型和LLM设置中的定义。

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Settings

documents = SimpleDirectoryReader('data').load_data()
nodes = Settings.node_parser.get_nodes_from_documents(documents)

# 设置嵌入模型和大型语言模型

Gemini 有各种型号，包括 gemini-pro、gemini-1.0-pro、gemini-1.5、视觉模型等。这里，我们将使用默认模型并提供 Google API Key。对于 Gemini 中的嵌入模型，我们目前使用的是 embedding-001。

In [1]:
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini

Settings.embed_model = GeminiEmbedding(
    model_name="models/embedding-001", api_key=GOOGLE_API_KEY
)
Settings.llm = Gemini(api_key=GOOGLE_API_KEY)

# 定义Storage context，并存储数据

一旦数据被解析为节点，LlamaIndex 就会提供一个存储上下文，它提供默认的文档存储，用于存储数据的向量嵌入。此存储上下文将数据保留在内存中，以便以后对其进行索引。

In [ ]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

为了构建自定义检索器以执行混合搜索，我们需要创建两个索引。第一个可以执行向量搜索的向量索引，第二个可以执行关键字搜索的关键字索引。为了创建索引，我们需要存储上下文和节点文档，以及嵌入模型和LLM的默认设置。

In [ ]:
from llama_index.core import SimpleKeywordTableIndex, VectorStoreIndex

vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

# 构建自定义Retriever

要使用 LlamaIndex 构建用于混合搜索的自定义检索器，我们首先需要定义架构，尤其是配置合适的节点。需要矢量索引检索器和关键字检索器来执行混合搜索，是通过指定模式（AND 或 OR）来实现这两种检索器的组合，集成这两种技术以最大限度地减少幻觉。
       
一旦节点配置好后，我们就可以使用 vector 和 keyword 检索器查询每个节点 ID 的数据。然后，根据所选模式，最终确定自定义检索器。

In [ ]:
from llama_index.core import QueryBundle
from llama_index.core.schema import NodeWithScore

from llama_index.core.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KeywordTableSimpleRetriever,
)
from typing import List
class CustomRetriever(BaseRetriever):
    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        keyword_retriever: KeywordTableSimpleRetriever,
        mode: str = "AND") -> None:

        self._vector_retriever = vector_retriever
        self._keyword_retriever = keyword_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode
        super().__init__()
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        keyword_nodes = self._keyword_retriever.retrieve(query_bundle)
        vector_ids = {n.node.node_id for n in vector_nodes}
        keyword_ids = {n.node.node_id for n in keyword_nodes}
        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in keyword_nodes})
        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(keyword_ids)
        else:
            retrieve_ids = vector_ids.union(keyword_ids)
        retrieve_nodes = [combined_dict[r_id] for r_id in retrieve_ids]
        return retrieve_nodes

# 定义retriever

定义好自定义检索器类后，我们需要实例化检索器并合成查询引擎。响应合成器用于根据用户查询和给定的文本块集生成LLM响应。Response Synthesizer 的输出是一个 Response 对象，该对象将自定义检索器作为参数之一。

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=2)
keyword_retriever = KeywordTableSimpleRetriever(index=keyword_index)
# custom retriever => combine vector and keyword retriever
custom_retriever = CustomRetriever(vector_retriever, keyword_retriever)
# define response synthesizer
response_synthesizer = get_response_synthesizer()
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

# 运行自定义检索查询引擎

最后，我们已经开发好了自定义的retriever，它可以显著减少幻觉。为了测试其有效性，我们使用一个包括上下文的提示和一个不包括上下文的提示来评估生成的响应。

In [ ]:
query = "what does the data context contain?"
print(custom_query_engine.query(query))
print(custom_query_engine.query("what is science?")